In [14]:
#Homework2,3(belief propagtion and state evolution)
import numpy as np
import scipy.stats as stats
import math
from numpy import linalg as LA
#import matplotlib.pyplot as plt

n = 100 #dimension
m = 10 #data
mean1 = -20*np.ones(n)/np.sqrt(n)#s_0 = [20,...,20]
mean2 = 20*np.ones(n)/np.sqrt(n)
cov = np.identity(n)
y = 0.5*np.random.multivariate_normal(mean1, cov, m) + 0.5*np.random.multivariate_normal(mean2, cov, m)
x = y+np.random.multivariate_normal(np.zeros(n), cov, m)
#print(y,y.shape)
    
#print(x)
#print(np.mean(x),np.var(x))
chi = 2*np.ones((m,n))
s = 20*np.ones((m,n))
p = np.ones((m,n))
W = 2*np.ones(m)
alpha = m/n
lamb = 2

def update_p_W_2(x,chi,s):
    for u in range(m):
        sum_W = 0
        for i in range(n):
            x_ = np.append(x[u][0:i],x[u][i+1:n])
            s_ = np.append(s[u][0:i],s[u][i+1:n])
            p[u][i] = x_@s_/np.sqrt(n)
            sum_W += x[u][i]**2*chi[u][i]/n
        W[u] = sum_W
    return p,W

def y_s(p,t):
    eps = 0.001
    y_st = 1
    flag =True
    while flag:
        y_new = np.tanh(p+np.sqrt(t)*y_st)*np.sqrt(t)
        if LA.norm(y_new - y_st) < eps:
            flag = False
            return y_new
            break
        y_st = y_new

def F(p,t):
    return y_s(p,t)/(np.sqrt(t))

def T(p,t):
    return (1-F(p,t)**2)/(1-t*(1-F(p,t)**2))

def update_chi_s_2(x,p,W):
    for i in range(n):
        t = np.zeros(m)
        f = np.zeros(m)
        xx = np.zeros(m)
        for j in range(m):
            t[j] = T(p[j][i],W[j])
            f[j] = F(p[j][i],W[j])
            xx[j] = x[j][i]**2
        for u in range(m):
            x_u = np.append(xx[0:u],xx[u+1:m])
            t_u = np.append(t[0:u],t[u+1:m])
            f_u = np.append(f[0:u],f[u+1:m])
            chi[u][i] = 1/(alpha + lamb - x_u@t_u/n)
            s[u][i] = chi[u][i]*np.sqrt(x_u)@f_u/np.sqrt(n)
    return chi,s


#testing
eps = 0.001
iter_ = 0

while iter_ <100:
    p_d = update_p_W_2(x,chi,s)[0]
    W_d = update_p_W_2(x,chi,s)[1]
    chi_d = update_chi_s_2(x,p_d,W_d)[0]
    s_d = update_chi_s_2(x,p_d,W_d)[1]

    chi = chi_d
    s = s_d
    print(s[0][0])
    iter_ += 1

0.06590162554761962
-0.1267672703459044
-0.02513570104767556
0.11406420065901468


/var/folders/h_/mft4g8yj4p5b9mc1x71fgd680000gn/T/ipykernel_34453/610983092.py:42: RuntimeWarning: invalid value encountered in sqrt
  y_new = np.tanh(p+np.sqrt(t)*y_st)*np.sqrt(t)


KeyboardInterrupt: 

In [17]:
#State Evoluation
import numpy as np
from numpy import linalg as LA
mu = 0
sigma = 1
z = np.random.normal(mu,sigma,100) #sampling z
#initialize
k_t = 10
sigma_s2 = 1
sigma_t2 = 10
alpha = 0.01
lamb = 3
chi_t = 2
eps = 0.001
parameter_old = np.array([chi_t,k_t,sigma_t2])
parameter_new = np.array([0,0,0])
iter_ = 0


def y_s(p,t):#using iteration to solve y^*
    eps = 0.001 #epsilon
    y_st = 1 #initial y^*
    flag =True
    while flag:
        y_new = np.tanh(p+np.sqrt(t)*y_st)*np.sqrt(t)
        if LA.norm(y_new - y_st) < eps: #update until threshold met
            flag = False
            return y_new #return the solution
            break
        y_st = y_new

def F(p,t): #F function
    return y_s(p,t)/(np.sqrt(t))

def T(p,t): #T function
    return (1-F(p,t)**2)/(1-t*(1-F(p,t)**2))

while LA.norm(parameter_new - parameter_old)>eps:
    parameter_old = np.array([chi_t,k_t,sigma_t2])
    z = np.random.normal(mu,sigma,100)
    sumz = 0
    sumf = 0
    for i in range(100):
        y = 0
        y = np.tanh(k_t*sigma_s2+np.sqrt(k_t**2*sigma_s2+sigma_t2)*z[i]+np.sqrt(chi_t)*y)*np.sqrt(chi_t)
        sumz += y/np.sqrt(chi_t)
        sumf += (y/np.sqrt(chi_t))**2
    f = sumz/100
    f2 = sumf/100
    t = (1-f**2)/(1-chi_t*(1-f**2))
    chi_new = 1/(alpha + lamb -alpha*t)
    k_new = alpha*chi_new*f
    sigma_new = alpha*chi_new**2*f2
    parameter_new = np.array([chi_new,k_new,sigma_new])
    chi_t = chi_new
    k_t = k_new
    sigma_t2 = sigma_new
    iter_+=1
    print("iteration", iter_, "paraUpdate", parameter_new)  

iteration 1 paraUpdate [0.317036   0.00220116 0.00095526]
iteration 2 paraUpdate [ 3.33849911e-01 -5.51509827e-06  7.91582125e-07]
iteration 3 paraUpdate [ 3.33891113e-01 -1.67673060e-07  8.45430368e-10]


In [20]:
#State Evoluation
import numpy as np
from numpy import linalg as LA
mu = 0
sigma = 1
z = np.random.normal(mu,sigma,100) #sampling z
#initialize
k_t = 1
sigma_s2 = 1
sigma_t2 = 10
alpha = 0.01
lamb = 3
chi_t = 2
eps = 0.001
parameter_old = np.array([chi_t,k_t,sigma_t2])
parameter_new = np.array([0,0,0])
iter_ = 0


def y_s(p,t):#using iteration to solve y^*
    eps = 0.001 #epsilon
    y_st = 1 #initial y^*
    flag =True
    while flag:
        y_new = np.tanh(p+np.sqrt(t)*y_st)*np.sqrt(t)
        if LA.norm(y_new - y_st) < eps: #update until threshold met
            flag = False
            return y_new #return the solution
            break
        y_st = y_new

def F(p,t): #F function
    return y_s(p,t)/(np.sqrt(t))

def T(p,t): #T function
    return (1-F(p,t)**2)/(1-t*(1-F(p,t)**2))

while LA.norm(parameter_new - parameter_old)>eps:
    parameter_old = np.array([chi_t,k_t,sigma_t2])
    z = np.random.normal(mu,sigma,100)
    sum_t = 0
    sum_f = 0
    sum_f2 = 0
    for i in range(100):
        sum_t += T(k_t*sigma_s2+np.sqrt(k_t**2*sigma_s2+sigma_t2)*z[i],chi_t)
        sum_f +=F(k_t*sigma_s2+np.sqrt(k_t**2*sigma_s2+sigma_t2)*z[i],chi_t)
        sum_f2 += F(k_t*sigma_s2+np.sqrt(k_t**2*sigma_s2+sigma_t2)*z[i],chi_t)**2
    chi_new = 1/(alpha + lamb -alpha*sum_t/100)
    k_new = alpha*chi_new*sum_f/100
    sigma_new = alpha*chi_new**2*sum_f2/100
    parameter_new = np.array([chi_new,k_new,sigma_new])
    chi_t = chi_new
    k_t = k_new
    sigma_t2 = sigma_new
    iter_+=1
    print("iteration", iter_, "paraUpdate", parameter_new)  

iteration 1 paraUpdate [0.3322556  0.00117035 0.00108493]
iteration 2 paraUpdate [3.33881319e-01 1.81068540e-05 2.59422327e-06]
iteration 3 paraUpdate [3.33891178e-01 2.49186403e-06 6.34474144e-09]
